In [1]:
# Append system path to include the config scripts
import sys
import os
from copy import deepcopy

print('importing lanfactory')
import lanfactory

print('importing ssms')
import ssms

print('importing hddm')
import hddm

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from config import *
#from model_performance_utils import *

import tensorflow
import torch
import config

import pandas as pd

importing lanfactory
importing ssms
importing hddm
importing lanfactory
importing ssms
importing hddm


In [ ]:
# Load basic dataframe

In [6]:
y = pickle.load(open('/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/ddm/subj_10_trials_1000/ddm_parameter_recovery_run_config.pickle', 'rb'))

In [5]:
my_config_generator.make_config_files(make_config_param_recov = 0,
                                      make_config_data_gen = 1,
                                      make_config_network = 1)

Making config for data generator
Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files
Moving on...
Saved to: 
/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/ddm_nsim_2000_dt_0.001_nps_100_npts_2000.pickle
Making config for networks
NEW PRINT
0
NEW PRINT
1
NEW PRINT
2
NEW PRINT
3
NEW PRINT
4
NEW PRINT
5
NEW PRINT
6
NEW PRINT
7
NEW PRINT
8
Now saving
/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/torch_network_train_config_ddm_nsim_2000_dt_0.001_nps_100_npts_2000_architecture_search.pickle


# DATA GENERATOR CONFIGS
#### Note: Look into the ssms package documentation to get a better idea about the kinds of configs that you need for different kinds of training data


In [3]:
ssms.config.data_generator_config['lan']['mlp']

{'output_folder': 'data/lan_mlp/',
 'dgp_list': 'ddm',
 'nbins': 0,
 'n_samples': 100000,
 'n_parameter_sets': 10000,
 'n_parameter_sets_rejected': 100,
 'n_training_samples_by_parameter_set': 1000,
 'max_t': 20.0,
 'delta_t': 0.001,
 'pickleprotocol': 4,
 'n_cpus': 'all',
 'kde_data_mixture_probabilities': [0.8, 0.1, 0.1],
 'simulation_filters': {'mode': 20,
  'choice_cnt': 0,
  'mean_rt': 17,
  'std': 0,
  'mode_cnt_rel': 0.9},
 'negative_rt_cutoff': -66.77497,
 'n_subruns': 10,
 'bin_pointwise': False,
 'separate_response_channels': False}

### Define Metadata

In [4]:
# Specify model
model = 'ddm_mic2_adj_gamma_conflict_no_bias'

# Where do you want to save the config file?
config_save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/'

# What kind of likelihood approximator are we generating training data for?
generator_approach = 'lan'

# Specific network type we train? (Might affect training data representation needed)
generator_network_type = 'mlp'

# Specify arguments which you want to adjust in the data generator
data_generator_arg_dict = {'dgp_list': model,
                           'n_samples': 200000,
                           'n_parameter_sets': 5000,
                           'delta_t': 0.001,
                           'n_training_samples_by_parameter_set': 2000,
                           'n_subruns': 5}

# model_config_arg_dict = {'param_bounds': [[-2.5, 0.2, 0.1, 0.0],
#                                           [2.5, 2.2, 0.9, 2.0]]}
model_config_arg_dict = {}

# Name of the config file
data_config_save_name = model + '_nsim_' + str(data_generator_arg_dict['n_samples']) + \
                        '_dt_' + str(data_generator_arg_dict['delta_t']) + \
                        '_nps_' + str(data_generator_arg_dict['n_parameter_sets']) + \
                        '_npts_' + str(data_generator_arg_dict['n_training_samples_by_parameter_set']) + '.pickle'

### Generate the Config File

In [5]:
make_data_generator_configs(model = model,
                            generator_approach = 'lan',
                            generator_network_type = 'mlp',
                            data_generator_arg_dict = data_generator_arg_dict,
                            model_config_arg_dict = model_config_arg_dict,
                            save_name = data_config_save_name,
                            save_folder = config_save_folder)

Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files
Moving on...
Saved to: 
/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/ddm_mic2_adj_gamma_conflict_no_bias_nsim_200000_dt_0.001_nps_5000_npts_2000.pickle


{'model_config': {'name': 'ddm_mic2_adj_gamma_conflict_no_bias',
  'params': ['vh',
   'vl1',
   'vl2',
   'd',
   't',
   'a',
   'theta',
   'scale',
   'alpha_gamma',
   'scale_gamma'],
  'param_bounds': [[-4.0, -4.0, -4.0, 0.0, 0.0, 0.3, 0.0, 0.0, 1.1, 0.5],
   [4.0, 4.0, 4.0, 1.0, 2.0, 2.5, 0.5, 5.0, 5.0, 5.0]],
  'boundary': <function ssms.basic_simulators.boundary_functions.gamma_conflict_bound(t=array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8

# NETWORK AND TRAIN CONFIGS

### Define Metadata

In [3]:
# Where do you want to save config files?
network_train_config_save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/'

# Specify training data folder:
training_data_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000'

# Provide a unique identifier for the particular files you need from the training_data_folder
# Sometimes the model name is not enough, since some model names form part of other model names
training_file_identifier = 'mic2_adj_no_bias' #'ddm_par2_angle_no_bias'

# Specify the name of the config file
network_train_config_save_name = 'network_train_config_' + training_file_identifier + '_nsim_200000_dt_0.001_nps_5000_npts_2000_architecture_search.pickle'

# How many epochs to train?
n_epochs = 20

# Network architectures
layer_sizes = [[100, 100, 100, 1], [100, 100, 100, 100, 1], [100, 100, 100, 100, 100, 1],
               [120, 120, 120, 1], [120, 120, 120, 120, 1], [120, 120, 120, 120, 120, 1],
               [150, 150, 150, 1], [150, 150, 150, 150, 1], [150, 150, 150, 150, 150, 1]]
layer_types = [['dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense', 'dense'],
               ['dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense', 'dense'],
               ['dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense'], ['dense', 'dense', 'dense', 'dense', 'dense', 'dense'],
              ]
activations = [['tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear'],
               ['tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear'],
               ['tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'linear'], ['tanh', 'tanh', 'tanh', 'tanh', 'tanh', 'linear'],
              ]

# Train / validations split
train_val_split = [0.98, 0.98, 0.98, 
                   0.98, 0.98, 0.98,
                   0.98, 0.98, 0.98]

[100]
config for gpu --> use large batch size


### Create the Config file

In [4]:
# Loop objects
config_dict = {}
network_arg_dicts = {}
train_arg_dicts = {}    
cnt = 0

for i in range(len(layer_sizes)):
    for j in range(len(n_training_files)):
        # Specify the arguments which you want to adjust in the network and train configs
        # Check: lanfactory.config.network_config_mlp
        #        lanfactor.config.train_config_mlp for details

        network_arg_dict = {'layer_types': layer_types[i],
                            'layer_sizes': layer_sizes[i],
                            'activations': activations[i],
                            'loss': ['huber'],
                            'model_id': training_file_identifier
                            }

        train_arg_dict = {'n_epochs': n_epochs,
                          'n_training_files': n_training_files[j],
                          'train_val_split': train_val_split[i],
                          'cpu_batch_size': 10000,
                          'gpu_batch_size': 100000,
                          'shuffle_files': True,
                          'label_prelog_cutoff_low': 1e-7,
                          'label_prelog_cutoff_high': None,
                          'save_history': True,
                          'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
                          }

        config_dict[cnt] = make_train_network_configs(training_data_folder=traning_data_folder,
                                                      training_file_identifier=training_file_identifier,
                                                      save_folder = network_train_config_save_folder,
                                                      train_val_split=train_val_split[i],
                                                      network_arg_dict = network_arg_dict,
                                                      train_arg_dict = train_arg_dict,
                                                      save_name = None)
        
        print('NEW PRINT')
        print(cnt)
        cnt += 1

print('Now saving')
pickle.dump(config_dict, open(network_train_config_save_folder + network_train_config_save_name, 'wb'))
print(network_train_config_save_folder + network_train_config_save_name)

NEW PRINT
0
NEW PRINT
1
NEW PRINT
2
NEW PRINT
3
NEW PRINT
4
NEW PRINT
5
NEW PRINT
6
NEW PRINT
7
NEW PRINT
8
Now saving
/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/gpu_torch_network_train_config_mic2_adj_no_bias_nsim_200000_dt_0.001_nps_5000_npts_2000_architecture_search.pickle


In [5]:
x = pickle.load(open(network_train_config_save_folder + network_train_config_save_name, 'rb'))

### MAKE PARAMETER RECOVERY CONFIG

In [16]:
from hddm.simulators.hddm_dataset_generators import simulator_h_c

def make_parameter_recovery_dataset(model = 'angle',
                                    save_file = '',
                                    save_folder = '',
                                    n_subjects = 10,
                                    n_trials_per_subject = 1000,
                                    n_datasets = 100):
    
    data_dict = {}
    for i in range(n_datasets):
        # MAKE DATA
        data, parameter_dict = simulator_h_c(data = None,
                                             n_subjects = n_subjects,
                                             n_trials_per_subject = n_trials_per_subject,
                                             model = model,
                                             p_outlier = 0.0,
                                             conditions = None,
                                             depends_on = None,
                                             regression_models = None,
                                             regression_covariates = None,
                                             group_only_regressors = True,
                                             group_only = None,
                                             fixed_at_default = None)

        data_dict[i] = {'data': data,
                        'parameter_dict': parameter_dict}
        
        if not i % 10:
            print(str(i), ' of ', str(n_datasets), ' finished')
    
    # Create save_folder if not already there
    lanfactory.utils.try_gen_folder(folder = save_folder, 
                                    allow_abs_path_folder_generation = True)
    
    if save_file != '':
        print('Saving...')
        pickle.dump(data_dict, open(save_folder + save_file, 'wb'))
        print('Saved to ', save_folder + save_file)
        
    return data_dict

### MAKE PARAMETER RECOVERY DATASET

In [4]:
# Speficy configs for parameter recovery dataset
model = 'angle'
n_data_sets = 100
n_subjects = 10
n_trials_per_subject = 1000

save_folder  = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/' + model + '/' + \
                'subj_' + str(n_subjects) + '_trials_' + str(n_trials_per_subject)
save_file_name = '/' + model + '_parameter_recovery_base_dataset_subj_' + str(n_subjects) + '_ntrials_' + str(n_trials_per_subject) + '.pickle'
#save_file_param_recov_base = save_folder + save_file_name

In [5]:
data_dict = make_parameter_recovery_dataset(model = model,
                                            save_folder = save_folder,
                                            save_file = save_file_name,
                                            #save_file = save_file_param_recov_base,
                                            n_subjects = n_subjects,
                                            n_trials_per_subject = n_trials_per_subject)

NameError: name 'make_parameter_recovery_dataset' is not defined

### MAKE PARAMETER RECOVERY CONFIG

In [6]:
dir(model_performance_utils)

NameError: name 'model_performance_utils' is not defined

In [7]:
import model_performance_utils
model = 'angle'
filter_ = 'training_data_angle'
network_data = model_performance_utils.get_model_performance_summary_df(filter_ = filter_)

# Get top n models here
n_lans_to_pick = 10
lan_files = network_data.loc[:, 'model_path'].to_list()[:n_lans_to_pick]
lan_config_files = network_data.loc[:, 'network_config_path'].to_list()[:n_lans_to_pick]
lan_ids = network_data.loc[:, 'model_id'].to_list()[:n_lans_to_pick]

# PARAMETER RECOVERY HYPERPARAMETERS

# Dataset specifics (for now basic hierarchical)
n_subjects = 10
n_trials_per_subject = 1000

# MCMC specifics
n_burn = 1000
n_mcmc = 4000
n_chains = 2

# Other metadata
#n_parameter_sets = 100
model_config =  deepcopy(hddm.model_config.model_config[model]) # load from

# Get date and model_file
# save_folder = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/' + model + '/' + \
#                 'subj_' + str(n_subjects) + '_trials_' + str(n_trials_per_subject) 


Overall val loss statistics: 

val loss mean:  0.0024939945691989
val loss min:  0.0022585825063288
val loss max:  0.0028880259487777


Top five network architectures 

   val_loss  n_hidden_layers  size_hidden_layers
7  0.002259                5                 100
1  0.002339                4                 100
0  0.002349                4                 120
6  0.002393                5                 120
8  0.002505                5                 150
2  0.002535                4                 150
5  0.002565                3                 150
3  0.002612                3                 120
4  0.002888                3                 100


Top five networks, paths 



In [8]:
make_param_recovery_configs(model_name = model,
                           parameter_recovery_data_loc = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/' + model + '/' + \
                                                            'subj_' + str(n_subjects) + '_trials_' + str(n_trials_per_subject) + '/' + model + \
                                                                '_parameter_recovery_base_dataset_subj_' + str(n_subjects) + \
                                                                    '_ntrials_' + str(n_trials_per_subject) + '.pickle',
                           lan_files = lan_files,
                           lan_ids = lan_ids,
                           lan_config_files = lan_config_files,
                           save_folder = save_folder,
                           model_config = model_config,
                           n_burn = n_burn,
                           n_mcmc = n_mcmc,
                           n_chains = n_chains)

Found folder:  /users
Moving on...
Found folder:  /users/afengler
Moving on...
Found folder:  /users/afengler/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/angle
Moving on...
Found folder:  /users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/angle/subj_10_trials_1000
Moving on...
Saving to: 
/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/angle/subj_10_trials_1000/angle_parameter_recovery_run_config.pickle


{'parameter_recovery_data_loc': '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/angle/subj_10_trials_1000/angle_parameter_recovery_base_dataset_subj_10_ntrials_1000.pickle',
 'model_config': {'doc': 'Model formulation is described in the documentation under LAN Extension.\nMeant for use with the extension.',
  'params': ['v', 'a', 'z', 't', 'theta'],
  'params_trans': [0, 0, 1, 0, 0],
  'params_std_upper': [1.5, 1.0, None, 1.0, 1.0],
  'param_bounds': [[-3.0, 0.3, 0.2, 0.001, -0.1], [3.0, 2.0, 0.8, 2.0, 1.45]],
  'param_bounds_cnn': [[-2.5, 0.2, 0.1, 0.0, 0.0],
   [2.5, 2.0, 0.9, 2.0, 1.3707963267948966]],
  'boundary': <function hddm.simulators.boundary_functions.angle(t=1, theta=1)>,
  'params_default': [0.0, 1.0, 0.5, 0.001, 0.0],
  'hddm_include': ['z', 'theta'],
  'choices': [-1, 1],
  'slice_widths': {'v': 1.5,
   'v_std': 1,
   'a': 1,
   'a_std': 1,
   'z': 0.1,
   'z_trans': 0.2,
   't': 0.01,
   't_std': 0.15,
   'theta': 0.1,
   'theta_std': 0.2}}

In [61]:
y = pickle.load(open( '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/parameter_recovery/angle/angle_parameter_recovery_config.pickle', 'rb'))

In [62]:
y

{'n_subjects': 10,
 'trials_per_subject': 1000,
 'model_config': {'doc': 'Model formulation is described in the documentation under LAN Extension.\nMeant for use with the extension.',
  'params': ['v', 'a', 'z', 't', 'theta'],
  'params_trans': [0, 0, 1, 0, 0],
  'params_std_upper': [1.5, 1.0, None, 1.0, 1.0],
  'param_bounds': [[-3.0, 0.3, 0.2, 0.001, -0.1], [3.0, 2.0, 0.8, 2.0, 1.45]],
  'param_bounds_cnn': [[-2.5, 0.2, 0.1, 0.0, 0.0],
   [2.5, 2.0, 0.9, 2.0, 1.3707963267948966]],
  'boundary': <function hddm.simulators.boundary_functions.angle(t=1, theta=1)>,
  'params_default': [0.0, 1.0, 0.5, 0.001, 0.0],
  'hddm_include': ['z', 'theta'],
  'choices': [-1, 1],
  'slice_widths': {'v': 1.5,
   'v_std': 1,
   'a': 1,
   'a_std': 1,
   'z': 0.1,
   'z_trans': 0.2,
   't': 0.01,
   't_std': 0.15,
   'theta': 0.1,
   'theta_std': 0.2}},
 'model_name': 'angle',
 'n_parameter_sets': 100,
 'lan_files': ['/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/torch_models/248c94cca33e11ecb